In [0]:

from delta.tables import DeltaTable

stage_bookings=spark.table("zoom_car.default.staging_bookings_delta")
stage_customers=spark.table("zoom_car.default.staging_customers_delta")
target_bookings="zoom_car.default.target_bookings_delta"
target_customers="zoom_car.default.target_customers_delta"

table_b=spark._jsparkSession.catalog().tableExists(target_bookings)
table_c=spark._jsparkSession.catalog().tableExists(target_customers)

if table_b:
    merged_bookings=DeltaTable.forName(spark,target_bookings)
    merged_bookings.alias("target")\
    .merge(stage_bookings.alias("updates"),
    "target.booking_id=updates.booking_id")\
    .whenMatchedDelete(condition="updates.status = 'cancelled'") \
    .whenMatchedUpdateAll(condition="updates.status != 'cancelled'")\
    .whenNotMatchedInsertAll().execute()
else:
    stage_bookings.write.format("delta").option("overwriteSchema","true").saveAsTable(target_bookings)

if table_c:
    merged_customers=DeltaTable.forName(spark,target_customers)
    merged_customers.alias("target")\
    .merge(stage_customers.alias("updates"),
    "target.customer_id=updates.customer_id")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll().execute()
else:
    stage_customers.write.format("delta").option("overwriteSchema","true").saveAsTable(target_customers)
#print(merged_bookings)




